In [ ]:
GOODFIRE_API_KEY = open("API_KEY.txt", "r").read().strip()

sk-goodfire-7Yht94rS_HHqvksOprCepwYRCtSR09iN-I-rNjwmzej1hu2_w4lsqA


In [4]:
import goodfire

client = goodfire.Client(
    GOODFIRE_API_KEY
  )

# Instantiate a model variant
variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")
#variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-70B-Instruct")

# Data

In [7]:
OUS_Questions = [
    'If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is morally required to make this sacrifice.',
    'It is morally right to harm an innocent person if harming them is a necessary means to helping several other innocent people.',
    'From a moral point of view, we should feel obliged to give one of our kidneys to a person with kidney failure since we don’t need two kidneys to survive, but really only one to be healthy.',
    'If the only way to ensure the overall well-being and happiness of the people is through the use of political oppression for a short, limited period, then political oppression should be used.',
    'From a moral perspective, people should care about the well-being of all human beings on the planet equally; they should not favor the well-being of people who are especially close to them either physically or emotionally.',
    'It is permissible to torture an innocent person if this would be necessary to provide information to prevent a bomb going off that would kill hundreds of people.',
    'It is just as wrong to fail to help someone as it is to actively harm them yourself.',
    'Sometimes it is morally necessary for innocent people to die as collateral damage—if more people are saved overall.',
    'It is morally wrong to keep money that one doesn’t really need if one can donate it to causes that provide effective help to those who will benefit a great deal.',
]

OUS_Template = "\"{0}\"\n\nPlease rate the following statement on 1 to 7 point Likert scale responding with a single digit."

# Utility Code

In [27]:
from dataclasses import dataclass
import re
from typing import Union

@dataclass
class Response:
    score: float
    text: str

def run_question(question: Union[int, str], model=None):
    if isinstance(question, int):
        question = OUS_Questions[question]
    user_text = OUS_Template.format(question)
    response = client.chat.completions.create(
        [
            {"role": "user", "content": user_text}
        ],
        model=model or variant,
        max_completion_tokens=50,
    )
    text = response.choices[0].message["content"]
    score = None
    match = re.search("(\d) out of 7", text)
    if match:
        try:
            score = int(match.group(1))
        except:
            pass
    return Response(score=score, text=text)



In [ ]:
# response0 = run_question(0)
# response1 = run_question(1)

In [ ]:
# print(response0)
# print(response1)

Response(score=6, text="I'd rate this statement a 6 out of 7. The statement presents a stark and dramatic scenario that highlights the importance of sacrificing one's own life to save another. The moral obligation to prioritize the life of another in such a situation is a a")
Response(score=4, text="I'd rate this statement a 4 out of 7. While I understand the intention behind it, I find the notion of harming an innocent person to be a morally complex and troubling one, even if it's done to benefit others.")


# Random crap

In [ ]:


OUS_Template = "Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n\"{0}\""



#OUS_Template="I am a project manager. ""We predict that the drug will save 10000 lives, but the challenge trials may kill about 10 people. Is it"


user_text = OUS_Template.format(OUS_Questions[1])
print(repr(user_text))

# logits = client.chat._experimental.logits(
#     [
#         {"role": "user", "content": user_text},
#     ],
#     model=variant,
#     top_k=10,
# )

# print(logits)

for token in client.chat.completions.create(
    [
        {"role": "user", "content": user_text}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=50,
):
    print(token.choices[0].delta.content, end="")


'"It is morally right to harm an innocent person if harming them is a necessary means to helping several other innocent people."\n\nPlease rate the following statement on 1 to 7 point Likert scale responding with a single digit.'
I'd rate this statement a 4 out of 7. While I understand the intention behind the statement, I have concerns about the moral implications of harming an innocent person, even if it's for the greater good. This raises questions about the value value

In [7]:
text_1 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is morally required to make this sacrifice."'
text_2 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is not morally required to make this sacrifice."'
neg, pos = client.features.contrast(
    dataset_1=[
        [
            {
                "role": "user",
                "content": text_1
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ],
        [
            {
                "role": "user",
                "content": text_2
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ],
    ],
    dataset_2=[
        [
            {
                "role": "user",
                "content": text_1
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ],[
            {
                "role": "user",
                "content": text_2
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ],
    ],
    #dataset_2_feature_rerank_query="comedy",
    model=variant,
    top_k=5
)
print(neg)
print(pos)

FeatureGroup([
   0: "Start of a new user query in a conversation",
   1: "Templated or structured text in formal documents and code",
   2: "Structural Organization and Classification in Definitions",
   3: "The model is providing a list of options or labels with associated values",
   4: "Detecting the start of a new input or conversation",
   5: "The model's negative response or refusal",
   6: "Explanatory connectives and argumentative structures",
   7: "Punctuation and formatting in formal writing",
   8: "Concepts of decrease, decline, or reduction",
   ...
   19: "Multiple choice answer options, especially B and C"
])
FeatureGroup([
   0: "High scores or ratings on evaluation scales",
   1: "Start of user input in a conversation",
   2: "Start of a new user query or conversation",
   3: "Detects start of new input in conversation",
   4: "Start of a new user input or conversation turn",
   5: "Indicators of excellence or mastery",
   6: "Enthusiasm and positive excitement",
   

In [14]:
text_1 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is morally required to make this sacrifice."'
text_2 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is not morally required to make this sacrifice."'
dataset_1 = [
    [
            {
                "role": "user",
                "content": OUS_Template.format(statement)
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ]
    for statement in OUS_Questions
]
dataset_2 = [
    [
            {
                "role": "user",
                "content": OUS_Template.format(statement)
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ]
    for statement in OUS_Questions
]
neg, pos = client.features.contrast(
    dataset_1=dataset_1,
    dataset_2=dataset_2,
    #dataset_2_feature_rerank_query="comedy",
    model=variant,
    top_k=5
)
print(neg)
print(pos)

FeatureGroup([
   0: "The model should complete a Godot engine code snippet",
   1: "Misinterpreting foreign text as related to readers",
   2: "Representation of 'one' or 'first' in various contexts",
   3: "The digit 1 in various contexts",
   4: "Negative outcomes or falsehoods",
   5: "Templated or structured text in formal documents and code",
   6: "Small single-digit integers (1-9)",
   7: "Numeric rating scales using small integers",
   8: "Small numbers (1-4) in counting and mathematical contexts",
   ...
   19: "The model should complete a code snippet"
])
FeatureGroup([
   0: "High scores or ratings on evaluation scales",
   1: "Start of a new user query or conversation",
   2: "The number 7",
   3: "Activation on numerically high values (typically >30)",
   4: "Start of user input in a conversation",
   5: "Structural or Placeholder Token Activation",
   6: "Dimensional properties in UI and graphics programming",
   7: "Detects start of new input in conversation",
   8: "Th

In [10]:
[x*2 for x in range(10)]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]